In [1]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


In [2]:
!pip install python_dotenv

In [3]:
!pip install jupyter_bokeh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.7 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [4]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# Load environment variables
if not load_dotenv(find_dotenv()):
    print("Warning: .env file not found!")

# Retrieve API key securely
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    raise ValueError("Error: OPENAI_API_KEY is not set. Please check your .env file.")

openai.api_key = api_key

print("OpenAI API key successfully loaded.")


OpenAI API key successfully loaded.


In [ ]:
# def get_completion(prompt, model="gpt-3.5-turbo"):
#     messages = [{"role": "user", "content": prompt}]
#     response = openai.ChatCompletion.create(
#         model=model,
#         messages=messages,
#         temperature=0, # this is the degree of randomness of the model's output
#     )
#     return response.choices[0].message["content"]

In [ ]:
# def get_completion_from_messages(messages, model="gpt-4", temperature=0.7):
#     response = openai.ChatCompletion.create(
#         model=model,
#         messages=messages,
#         temperature=temperature, # this is the degree of randomness of the model's output

#     )
# #     print(str(response.choices[0].message))
#     return response.choices[0].message["content"]

In [ ]:
# messages =  [
# {'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},
# {'role':'user', 'content':'tell me a joke'},
# {'role':'assistant', 'content':'Why did the chicken cross the road'},
# {'role':'user', 'content':'I don\'t know'}  ]

# response = get_completion_from_messages(messages, temperature=1)
# # print(response)

In [ ]:
# messages =  [
# {'role':'system', 'content':'You are friendly chatbot.'},
# {'role':'user', 'content':'Hi, my name is Isa'}  ]
# response = get_completion_from_messages(messages, temperature=1)
# # print(response)

In [ ]:
# messages =  [
# {'role':'system', 'content':'You are friendly chatbot.'},
# {'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
# response = get_completion_from_messages(messages, temperature=1)
# # print(response)

In [ ]:
# messages =  [
# {'role':'system', 'content':'You are friendly chatbot.'},
# {'role':'user', 'content':'Hi, my name is Isa'},
# {'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
# Is there anything I can help you with today?"},
# {'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
# response = get_completion_from_messages(messages, temperature=1)
# # print(response)

In [5]:
import openai

def get_completion_from_messages(messages, model="gpt-4", temperature=0.7):
    """
    Generates a response from OpenAI's ChatCompletion model.

    Parameters:
        messages (list): A list of message dictionaries in the format [{"role": "user", "content": "Hello"}].
        model (str): The OpenAI model to use (default: "gpt-4").
        temperature (float): Controls randomness in responses (default: 0.7).

    Returns:
        str: The AI's response.
    """
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=temperature
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error in get_completion_from_messages: {e}")
        return "An error occurred while generating a response."


In [6]:
import panel as pn

def collect_messages(_):
    """
    Collects user input, appends it to the conversation context,
    generates a response, and updates the UI.
    """
    prompt = inp.value  # Get user input
    inp.value = ''  # Clear input field after submission

    # Append user input to context
    context.append({'role': 'user', 'content': prompt})

    # Generate AI response
    response = get_completion_from_messages(context)

    # Append AI response to context
    context.append({'role': 'assistant', 'content': response})

    # Update UI panels with User and Assistant messages
    panels.append(pn.Row("User:", pn.pane.Markdown(prompt, width=600)))

    # Assistant response with a styled background
    assistant_response = pn.pane.Markdown(
        f"**Assistant:** {response}",
        width=600,
        styles={'background-color': '#F6F6F6', 'padding': '5px', 'border-radius': '5px'}
    )
    panels.append(pn.Row(assistant_response))

    # Ensure conversation panel remains scrollable
    return pn.Column(
        *panels,
        height=400,  # Fixed height
        scroll=True   # Enables scrolling when content overflows
    )


In [7]:
import panel as pn  # GUI

# Initialize Panel
pn.extension()

# Collect display messages
panels = []

# Define System Context
context = [{
    'role': 'system',
    'content': """
You are an Everyday Conversational AI Agent for Children,
a friendly chatbot that interacts with kids about their daily experiences.
Start with a warm greeting to create a positive atmosphere.

### Response Guidelines:
- Speak as if the child is **5 years old**.
- Keep responses **short (under 50 words)**.
- Use **simple words** and **easy-to-understand sentences**.
- Provide **fun and relatable examples**.
- Avoid **harmful, rude, or age-inappropriate content**.
- If a child asks about **sensitive topics**, redirect with a **kind and convincing answer**.
- Break down **complex topics** into **smaller, understandable parts**.
- Ensure responses can be **read in under a minute**.
"""
}]

# User Input Field
inp = pn.widgets.TextInput(value="Hi", placeholder="Enter text here…")

# Button to Send Messages
button_conversation = pn.widgets.Button(name="Chat!")

# Bind Function to Button Click
interactive_conversation = pn.bind(collect_messages, button_conversation)

# Create Dashboard Layout
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

# Display Dashboard
dashboard


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)